# explorer

> CLI tool for API exploration

In [ ]:
#| default_exp cli.explorer

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, List, Dict, Tuple, Any
import typer
from typer import Argument, Option
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.text import Text
from rich.syntax import Syntax
import importlib
import inspect
import pkgutil
from pathlib import Path

## Core CLI Application

Create the main Typer application and console for rich output:

In [ ]:
#| export
app = typer.Typer(
    help="🎨 Tailwind CSS Explorer for FastHTML - Explore utilities dynamically",
    no_args_is_help=True,
    pretty_exceptions_enable=False
)
console = Console()

## Helper Functions

Utility functions for formatting output and discovering modules:

In [ ]:
#| export
def show_header():
    """Display the CLI header with welcome message."""
    header = Panel(
        "[bold cyan]🎨 Tailwind CSS Explorer for FastHTML[/bold cyan]\n"
        "[dim]Explore Tailwind CSS utilities dynamically from the cjm-fasthtml-tailwind library.[/dim]",
        expand=False,
        border_style="blue"
    )
    console.print(header)
    console.print()

In [ ]:
#| export
def discover_modules() -> Dict[str, List[str]]:
    """
    Dynamically discover all modules in the library.
    Returns a dictionary with categories as keys and module names as values.
    """
    modules_by_category = {}
    
    try:
        # Import the base package
        import cjm_fasthtml_tailwind
        
        # Get the package path
        package_path = Path(cjm_fasthtml_tailwind.__file__).parent
        
        # Iterate through subdirectories
        for category_dir in package_path.iterdir():
            if category_dir.is_dir() and not category_dir.name.startswith('_'):
                category = category_dir.name
                modules = []
                
                # Find Python files in this category
                for module_path in category_dir.glob('*.py'):
                    if not module_path.name.startswith('_'):
                        module_name = module_path.stem
                        modules.append(module_name)
                
                # Only add category if it has modules
                if modules:
                    modules_by_category[category] = sorted(modules)
        
    except Exception as e:
        console.print(f"[red]Error discovering modules: {e}[/red]")
        console.print("[yellow]Unable to discover modules dynamically.[/yellow]")
    
    return modules_by_category

In [ ]:
#| export
def get_module_info(module_name: str) -> Dict[str, Any]:
    """
    Get detailed information about a specific module.
    Returns dictionary with module details, utilities, and examples.
    """
    info = {
        "description": "No description available",
        "utilities": [],
        "examples": [],
        "imports": []
    }
    
    try:
        # First, find which category this module belongs to
        modules = discover_modules()
        category = None
        for cat, mods in modules.items():
            if module_name in mods:
                category = cat
                break
        
        if not category:
            info["description"] = f"Module '{module_name}' not found"
            return info
        
        # Import the module dynamically
        module_path = f"cjm_fasthtml_tailwind.{category}.{module_name}"
        module = importlib.import_module(module_path)
        
        # Get module description from docstring
        if module.__doc__:
            # Extract the first line of the docstring
            doc_lines = module.__doc__.strip().split('\n')
            if doc_lines:
                info["description"] = doc_lines[0].strip()
        
        # Find all exported items
        utilities = []
        imports = []
        
        # Get all module members
        for name, obj in inspect.getmembers(module):
            # Skip private members and imports
            if name.startswith('_'):
                continue
                
            # Check if it's a utility (has a build method or is a factory)
            if (hasattr(obj, 'build') or 
                hasattr(obj, '__call__') or 
                (inspect.isclass(obj) and name.endswith('Factory'))):
                
                # Skip test functions
                if not name.startswith('test_'):
                    utilities.append(name)
                    imports.append(name)
        
        info["utilities"] = sorted(utilities)
        
        # Find test functions to identify example categories
        test_functions = []
        for name, obj in inspect.getmembers(module):
            if name.startswith(f'test_{module_name}_') and callable(obj):
                # Extract the feature name from test_module_feature_examples
                parts = name.split('_')
                if len(parts) >= 3 and parts[-1] == 'examples':
                    feature = '_'.join(parts[2:-1])  # Join middle parts
                    test_functions.append((feature, name))
        
        # Format example categories
        if test_functions:
            for feature, func_name in sorted(test_functions):
                # Try to get description from function docstring
                func = getattr(module, func_name)
                desc = ""
                if func.__doc__:
                    desc = func.__doc__.strip().split('\n')[0]
                
                info["examples"].append(f"• {feature:<20} - {desc}")
        
        # Generate import statement
        if imports:
            import_list = sorted(imports)
            if len(import_list) <= 3:
                import_stmt = f"from {module_path} import {', '.join(import_list)}"
            else:
                # Multi-line import for many items
                import_items = ',\n    '.join(import_list)
                import_stmt = f"from {module_path} import (\n    {import_items}\n)"
            
            info["imports"] = import_stmt
            
            # Also add wildcard import option
            info["imports"] += f"\n\n# Or import everything:\nfrom {module_path} import *"
        
    except Exception as e:
        info["description"] = f"Error loading module '{module_name}': {str(e)}"
    
    return info

In [ ]:
#| export
def get_utility_info(module_name: str, utility_name: str) -> Dict[str, Any]:
    """
    Get detailed information about a specific utility.
    Returns dictionary with utility details, supported values, and usage.
    """
    # Placeholder implementation
    return {
        "description": f"[Placeholder: Description for {utility_name} utility]",
        "factory_type": "[Placeholder: Factory type information]",
        "supported_values": "[Placeholder: Supported values will be listed here]",
        "directional_variants": "[Placeholder: Directional variants if applicable]",
        "usage_example": f"[Placeholder: Usage example for {utility_name}]"
    }

In [ ]:
#| export
def run_example_function(module_name: str, feature: str) -> str:
    """
    Run a test example function and return its output.
    """
    # Placeholder implementation
    return f"[Placeholder: Output from test_{module_name}_{feature}_examples() will be shown here]"

In [ ]:
#| export
def search_utilities(search_term: str) -> List[Tuple[str, str, str]]:
    """
    Search for utilities across all modules.
    Returns list of (module, utility, description) tuples.
    """
    # Placeholder implementation
    return [
        ("module_name", "utility_name", "[Placeholder: Search results will be shown here]")
    ]

## Main Entry Point

The default command that shows when running the CLI without arguments:

In [ ]:
#| export
@app.callback(invoke_without_command=True)
def main(ctx: typer.Context):
    """
    Main entry point - shows header and quick start guide when no command is given.
    """
    if ctx.invoked_subcommand is None:
        show_header()
        
        # Quick start guide
        console.print("[bold]Quick Start Commands:[/bold]")
        console.print("  list              - Show all available modules")
        console.print("  list utilities    - Show all utilities with descriptions")
        console.print("  show <module>     - Explore a specific module (e.g., 'show spacing')")
        console.print("  search <term>     - Search for utilities (e.g., 'search margin')")
        console.print("  help              - Show detailed help")
        console.print()
        console.print("[dim]Example:[/dim]")
        console.print("  $ tw-explorer show spacing")
        console.print("  $ tw-explorer examples spacing basic")

## List Command

Command to list available modules and utilities:

In [ ]:
#| export
@app.command()
def list(
    target: Optional[str] = Argument(
        None,
        help="What to list: 'modules' (default), 'utilities', or 'all'"
    )
):
    """
    List available modules or utilities.
    
    Examples:
        tw-explorer list              # List all modules
        tw-explorer list utilities    # List all utilities
        tw-explorer list all          # List everything
    """
    show_header()
    
    if target is None or target == "modules":
        # List modules by category
        console.print("[bold]📦 Available Modules:[/bold]\n")
        
        modules = discover_modules()
        for category, module_list in modules.items():
            console.print(f"[bold cyan]{category.title()}:[/bold cyan]")
            for module in module_list:
                console.print(f"  • {module}")
            console.print()
        
        console.print("[dim]Use 'show <module>' to explore a specific module.[/dim]")
    
    elif target == "utilities":
        # List all utilities
        console.print("[bold]🔧 All Available Utilities:[/bold]\n")
        console.print("[Placeholder: Complete list of utilities will be shown here]")
        
    elif target == "all":
        # List everything
        console.print("[bold]📋 Complete Library Reference:[/bold]\n")
        console.print("[Placeholder: Complete reference will be shown here]")
        
    else:
        console.print(f"[red]Unknown target: {target}[/red]")
        console.print("Valid options: modules, utilities, all")

## Show Command

Command to show details about a module or utility:

In [ ]:
#| export
@app.command()
def show(
    module_name: str = Argument(..., help="Module name to explore"),
    utility_name: Optional[str] = Argument(None, help="Specific utility to examine")
):
    """
    Show detailed information about a module or specific utility.
    
    Examples:
        tw-explorer show spacing          # Show spacing module overview
        tw-explorer show spacing p        # Show details about 'p' utility
        tw-explorer show layout display   # Show details about display utilities
    """
    show_header()
    
    if utility_name is None:
        # Show module overview
        console.print(f"[bold]📐 Module: {module_name}[/bold]")
        console.print("=" * 30)
        
        info = get_module_info(module_name)
        
        # Description
        console.print(info["description"])
        console.print()
        
        # Utilities
        if info["utilities"]:
            console.print("[bold]🔧 Available Utilities:[/bold]")
            # Format utilities in columns
            utils = info["utilities"]
            for i in range(0, len(utils), 4):
                row = utils[i:i+4]
                formatted_row = "  " + ", ".join(f"[cyan]{u}[/cyan]" for u in row)
                console.print(formatted_row)
            console.print()
        
        # Example categories
        if info["examples"]:
            console.print("[bold]📚 Example Categories:[/bold]")
            for example in info["examples"]:
                console.print(example)
            console.print()
        
        # Commands
        console.print("[bold]💡 Commands:[/bold]")
        console.print(f"  examples {module_name} <feature>    - Run examples")
        console.print(f"  imports {module_name}               - Show import statement")
        console.print(f"  show {module_name} <utility>        - Show utility details")
        
    else:
        # Show specific utility
        console.print(f"[bold]🎯 Utility: {utility_name}[/bold]")
        console.print("=" * 30)
        
        info = get_utility_info(module_name, utility_name)
        
        # Description
        console.print(info["description"])
        console.print()
        
        # Factory type
        console.print(f"[bold]Factory Type:[/bold] {info['factory_type']}")
        console.print()
        
        # Supported values
        console.print("[bold]📊 Supported Values:[/bold]")
        console.print(info["supported_values"])
        console.print()
        
        # Directional variants
        if info["directional_variants"]:
            console.print("[bold]🧭 Directional Variants:[/bold]")
            console.print(info["directional_variants"])
            console.print()
        
        # Usage example
        console.print("[bold]💡 Usage Example:[/bold]")
        console.print(info["usage_example"])

## Examples Command

Command to run example functions and see their output:

In [ ]:
#| export
@app.command()
def examples(
    module_name: str = Argument(..., help="Module name"),
    feature: Optional[str] = Argument(None, help="Feature to demonstrate (e.g., 'basic', 'directional')")
):
    """
    Run example functions to see utility usage.
    
    Examples:
        tw-explorer examples spacing              # List available examples
        tw-explorer examples spacing basic        # Run basic spacing examples
        tw-explorer examples layout display       # Run display examples
    """
    show_header()
    
    if feature is None:
        # List available examples for the module
        console.print(f"[bold]📚 Available Examples for '{module_name}':[/bold]\n")
        
        info = get_module_info(module_name)
        console.print(info["examples"])
        console.print()
        console.print(f"[dim]Run 'examples {module_name} <feature>' to see specific examples.[/dim]")
        
    else:
        # Run specific example
        console.print(f"[bold]✨ Running: test_{module_name}_{feature}_examples()[/bold]\n")
        
        result = run_example_function(module_name, feature)
        
        # Display the results
        syntax = Syntax(result, "python", theme="monokai", line_numbers=False)
        console.print(Panel(syntax, title="Examples", border_style="green"))
        
        console.print("\n[green]✅ All examples passed![/green]")

## Search Command

Command to search for utilities across all modules:

In [ ]:
#| export
@app.command()
def search(
    term: str = Argument(..., help="Search term (e.g., 'margin', 'flex', 'grid')")
):
    """
    Search for utilities across all modules.
    
    Examples:
        tw-explorer search margin     # Find all margin-related utilities
        tw-explorer search flex       # Find flexbox utilities
        tw-explorer search negative   # Find utilities that support negative values
    """
    show_header()
    
    console.print(f"[bold]🔍 Search results for '[cyan]{term}[/cyan]':[/bold]\n")
    
    results = search_utilities(term)
    
    if not results:
        console.print(f"[yellow]No results found for '{term}'.[/yellow]")
        console.print("\n[dim]Try searching for:[/dim]")
        console.print("  • margin, padding, spacing")
        console.print("  • width, height, size")
        console.print("  • flex, grid, display")
        console.print("  • position, z-index")
        return
    
    # Display results in a table
    table = Table(title=f"Found {len(results)} results")
    table.add_column("Module", style="cyan")
    table.add_column("Utility", style="green")
    table.add_column("Description", style="dim")
    
    for module, utility, description in results:
        table.add_row(module, utility, description)
    
    console.print(table)
    console.print()
    console.print(f"[dim]💡 Use 'show {results[0][0]} {results[0][1]}' to explore a specific utility[/dim]")

## Imports Command

Command to show import statements for a module:

In [ ]:
#| export
@app.command()
def imports(
    module_name: str = Argument(..., help="Module name to get imports for")
):
    """
    Show import statements for a module.
    
    Examples:
        tw-explorer imports spacing      # Show spacing module imports
        tw-explorer imports layout       # Show layout module imports
    """
    show_header()
    
    console.print(f"[bold]📦 Import statements for '[cyan]{module_name}[/cyan]' module:[/bold]\n")
    
    info = get_module_info(module_name)
    
    # Display import statement
    syntax = Syntax(info["imports"], "python", theme="monokai", line_numbers=False)
    console.print(Panel(syntax, title="Import Statement", border_style="blue"))
    
    console.print("\n[dim]💡 Copy and paste the import statement into your code to use these utilities.[/dim]")

## Help Command

Enhanced help with usage examples:

In [ ]:
#| export
@app.command()
def help():
    """
    Show detailed help and usage guide.
    """
    show_header()
    
    help_text = """
[bold]📖 Detailed Usage Guide[/bold]

This CLI tool helps you explore the cjm-fasthtml-tailwind library dynamically.
All information is pulled directly from the library modules - nothing is hardcoded.

[bold cyan]Navigation Flow:[/bold cyan]
1. Start with 'list' to see available modules
2. Use 'show <module>' to explore a specific module
3. Run 'examples <module> <feature>' to see usage examples
4. Use 'search <term>' to find specific utilities

[bold cyan]Common Workflows:[/bold cyan]

[bold]🔍 Exploring a utility:[/bold]
  $ tw-explorer show spacing          # See what's in spacing module
  $ tw-explorer show spacing p        # Learn about padding utility
  $ tw-explorer examples spacing basic # See basic usage examples

[bold]🔎 Finding utilities:[/bold]
  $ tw-explorer search margin         # Find margin utilities
  $ tw-explorer list utilities        # See all available utilities

[bold]📝 Getting code:[/bold]
  $ tw-explorer imports spacing       # Get import statement
  $ tw-explorer examples spacing directional # See code examples

[bold cyan]Tips:[/bold cyan]
• Use tab completion for module and utility names
• Examples show actual output from the library's test functions
• All utilities are dynamically discovered - no hardcoded information

[dim]For more information, visit the project repository.[/dim]
"""
    
    console.print(help_text)

## CLI Entry Point

The main function to run the CLI:

In [ ]:
#| export
def cli():
    """Main CLI entry point."""
    app()

## Testing the CLI

Let's test some basic CLI functionality with the placeholder implementations:

In [ ]:
# Test showing the header
show_header()

╭────────────────────────────────────────────────────────────────────────────────────╮
│ 🎨 Tailwind CSS Explorer for FastHTML                                              │
│ Explore Tailwind CSS utilities dynamically from the cjm-fasthtml-tailwind library. │
╰────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Test discovering modules
modules = discover_modules()
print("Discovered modules:")
for category, module_list in modules.items():
    print(f"\n{category}:")
    for module in module_list:
        print(f"  - {module}")

Discovered modules:

builders:
  - scales

cli:
  - explorer

utilities:
  - flexbox_and_grid
  - layout
  - sizing
  - spacing

core:
  - base
  - resources
  - testing


In [ ]:
# Test getting module info
info = get_module_info("spacing")
print(f"Description: {info['description']}")
print(f"\nUtilities ({len(info['utilities'])}):")
for util in info['utilities'][:5]:  # Show first 5
    print(f"  - {util}")
print(f"\nExamples ({len(info['examples'])}):")
for ex in info['examples'][:3]:  # Show first 3
    print(ex)

Description: Padding and margin utilities for Tailwind CSS

Utilities (15):
  - DirectionalScaledFactory
  - Optional
  - ScaledFactory
  - SpaceFactory
  - TailwindScale

Examples (10):
• arbitrary            - Test padding utilities with arbitrary and custom values.
• basic                - Test basic padding utilities with various scale values.
• directional          - Test directional padding utilities.


## Export

Export the module for use:

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()